In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving insurance_data.csv to insurance_data.csv


In [ ]:
import io
df = pd.read_csv(io.BytesIO(uploaded['insurance_data.csv']))
df.head()

,Age,Affordability,Bought_insurance
0,22,1,0
1,25,0,0
2,47,1,1
3,52,0,0
4,46,1,1


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df[['Age', 'Affordability']], df.Bought_insurance, test_size = 0.2, random_state = 10)

**Scaling Data**

In [ ]:
X_train_scaled = X_train.copy()
X_train_scaled['Age'] = X_train_scaled['Age'] / 100

X_test_scaled = X_test.copy()
X_test_scaled['Age'] = X_test_scaled['Age'] / 100

**Finding Results for single layer neural network using Keras**

In [ ]:
model = keras.Sequential([
                          keras.layers.Dense(1, input_shape = (2, ), activation = 'sigmoid', kernel_initializer = 'ones', bias_initializer = 'zeros')
])

model.compile(
    optimizer = 'adam',
    loss = 'binary_crossentropy',
    metrics = ['accuracy']
)

model.fit(X_train_scaled, y_train, epochs = 5000)

Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 4ms/step - loss: 0.5596 - accuracy: 0.6818
Epoch 2039/5000
1/1 [==============================] - 0s 4ms/step - loss: 0.5595 - accuracy: 0.6818
Epoch 2040/5000
1/1 [==============================] - 0s 4ms/step - loss: 0.5595 - accuracy: 0.6818
Epoch 2041/5000
1/1 [==============================] - 0s 5ms/step - loss: 0.5594 - accuracy: 0.6818
Epoch 2042/5000
1/1 [==============================] - 0s 4ms/step - loss: 0.5594 - accuracy: 0.6818
Epoch 2043/5000
1/1 [==============================] - 0s 4ms/step - loss: 0.5594 - accuracy: 0.6818
Epoch 2044/5000
1/1 [==============================] - 0s 5ms/step - loss: 0.5593 - accuracy: 0.6818
Epoch 2045/5000
1/1 [==============================] - 0s 5ms/step - loss: 0.5593 - accuracy: 0.6818
Epoch 2046/5000
1/1 [==============================] - 0s 5ms/step - loss: 0.5592 - accuracy: 0.6818
Epoch 2047/5000
1/1 [==============================] - 0

In [ ]:
model.evaluate(X_test_scaled, y_test)

Storing the weights and bias obtained from training the Keras sequential single layer neural network

In [ ]:
coef, intercept = model.get_weights()

In [ ]:
coef, intercept

**Custom neural network implementation starts:**

Using sigmoid as the activation function


In [ ]:
def sigmoid(X):
  return (1 / (1 + np.exp(-X)))

Creating the logistic loss function (Binary cross entropy)


In [ ]:
def log_loss(y_true, y_pred): #binary_crossenropy
  eps = 1e-15
  y_pred2 = [max(min(i, 1 - eps), eps) for i in y_pred]
  y_pred2 = np.array(y_pred2)

  return -np.mean(y_true * np.log(y_pred2) + (1 - y_true) * np.log(1 - y_pred2))

Custom Neural network class with gradient descent, fit, and predict functions. (Initializing the weights as 1, bias as 0, and the learning rate as 0.5
)

In [ ]:
class NN:
  def __init__(self):
    self.w1 = 1
    self.w2 = 1
    self.bias = 0

  def gradient_descent(self, age, Affordability, y_true, epochs, loss_threshold):
    w1 = 1
    w2 = 2
    bias = 0
    rate = 0.5
    n = len(age)

    for i in range(epochs):
      weighted_sum = w1 * age + w2 * Affordability + bias      
      y_pred = sigmoid(weighted_sum)

      loss = log_loss(y_true, y_pred)

      w1d = (1 / n) * np.dot(np.transpose(age) , (y_pred - y_true))
      w2d = (1 / n) * np.dot(np.transpose(Affordability) , (y_pred - y_true))
      biasd = np.mean(y_pred - y_true)

      w1 = w1 - rate * w1d
      w2 = w2 - rate * w2d
      bias = bias - rate * biasd

      if i % 50 == 0:
        print(f'Epoch : {i}, w1 : {w1}, w2 : {w2}, bias : {bias}, loss : {loss}')

      if loss <= loss_threshold:
        break

    return w1, w2, bias

  def fit(self, X, y, epochs, loss_threshold):
    self.w1, self.w2, self.bias = self.gradient_descent(X['Age'], X['Affordability'], y, epochs, loss_threshold)

  def predict(self, X_test):
    weighted_sum = self.w1 * X_test['Age'] + self.w2 * X_test['Affordability'] + self.bias
    return sigmoid(weighted_sum)

**Training custom model till the loss threshold reaches the final loss for the tensorflow model**

In [ ]:
custom_nn = NN()
custom_nn.fit(X_train_scaled, y_train, epochs = 500, loss_threshold = 0.4680)

Epoch : 0, w1 : 0.954243381029754, w2 : 1.8934782723601609, bias : -0.16334188405356956, loss : 0.9079597418941617
Epoch : 50, w1 : 1.3286194033835557, w2 : 1.4469923843482826, bias : -1.5337158400562683, loss : 0.5619629044000221
Epoch : 100, w1 : 2.0054925839980675, w2 : 1.5672822901852925, bias : -1.91605828928374, loss : 0.537051127623175
Epoch : 150, w1 : 2.6288721622447944, w2 : 1.6289433284957722, bias : -2.225802441847183, loss : 0.5174274343829647
Epoch : 200, w1 : 3.1996387679364378, w2 : 1.6626492385484397, bias : -2.489327482514487, loss : 0.5015105259568468
Epoch : 250, w1 : 3.721022885786006, w2 : 1.6831836454804383, bias : -2.720788084294778, loss : 0.48842926610890525
Epoch : 300, w1 : 4.197150916026155, w2 : 1.6978509962932142, bias : -2.928335439941193, loss : 0.4775919949967547
Epoch : 350, w1 : 4.632335170179185, w2 : 1.7102443725045657, bias : -3.1169696244504226, loss : 0.4685569065738369


We can observe that the values of weights(w1, w2) and bias calculated by the tensorflow model and our custom neural network are very close by 

In [ ]:
coef, intercept

(array([[4.745832 ],
        [1.5486312]], dtype=float32), array([-3.0031297], dtype=float32))

**Prediction using custom model**

In [ ]:
custom_nn.predict(X_test_scaled)

7     0.417743
21    0.128049
5     0.767194
2     0.684091
13    0.144507
19    0.358850
dtype: float64

**Prediction using tensorflow model**

In [ ]:
model.predict(X_test_scaled)

array([[0.46117076],
       [0.14564016],
       [0.76908773],
       [0.68482655],
       [0.16426393],
       [0.3542868 ]], dtype=float32)

**We have achieved similar results to that of the tensorflow sequential single layered neural network**